# Conv2d MNIST Model
In this notebook we will create a model to predicts what letter is written in a 28x28 image based on the [MNIST dataset](http://yann.lecun.com/exdb/mnist/).  This is done via a convolutional Neural Network on the images.

### Import Tensorflow & Download data

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True, reshape=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Model

In [2]:
# HyperParams
learning_rate = 0.002
batch_size = 64
epochs = 10


# Input
inputs = tf.placeholder(tf.float32, [None, 28, 28, 1], name='inputs')
labels = tf.placeholder(tf.float32, [None, 10], name='labels')

is_training = tf.placeholder(tf.bool, name='is_training')

with tf.name_scope('Conv_1'):
    # Conv 1
    layer = tf.layers.conv2d(inputs, 16, 5, strides=2, padding='same', use_bias=False, activation=None)
    # layer = tf.layers.batch_normalization(layer, training=is_training)
    layer = tf.nn.relu(layer)
    # ?x14x14x16

with tf.name_scope('Conv_2'):
    # Conv 2
    layer = tf.layers.conv2d(layer, 32, 5, strides=2, padding='same', use_bias=False, activation=None)
    # layer = tf.layers.batch_normalization(layer, training=is_training)
    layer = tf.nn.relu(layer)
    # ?x7x7x32

    layer = tf.reshape(layer, (-1, 7*7*32))

with tf.name_scope('Dense_1'):
    # Fully Connected
    layer = tf.layers.dense(layer, 64, use_bias=False, activation=None)
    # layer = tf.layers.batch_normalization(layer, training=is_training)
    layer = tf.nn.relu(layer)
    # ?x64
    
with tf.name_scope('Logits'):
    # Logits
    logits = tf.layers.dense(layer, 10)
    # ?x10

with tf.name_scope('Loss'):
    # Loss
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))

with tf.name_scope('Optimizers'):
    # Optimizer
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        train_opt = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

with tf.name_scope('Accuracy'):
    # Accuracy
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    file_writer = tf.summary.FileWriter('./mnist-conv2d-bn-logs/1', sess.graph)

## Training

In [3]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for e in range(epochs):
        for ii in range(mnist.train.num_examples//batch_size):
            batch_inputs, batch_labels = mnist.train.next_batch(batch_size)
            
            sess.run(train_opt, feed_dict={
                inputs: batch_inputs, 
                labels: batch_labels, 
                is_training: True
            })
        
        b_loss, b_accuracy = sess.run([loss, accuracy], feed_dict={
            inputs: batch_inputs, 
            labels: batch_labels, 
            is_training: False
        })
        
        v_loss, v_accuracy = sess.run([loss, accuracy], feed_dict={
            inputs: mnist.validation.images, 
            labels: mnist.validation.labels, 
            is_training: False
        })
        
        print(
            "Epoch {:2d}/{}...".format(e+1, epochs),
            "Batch Loss: {:>3.5f}...".format(b_loss),
            "Batch Accuracy: {:>3.5f}...".format(b_accuracy),
            "Valid Loss: {:>3.5f}...".format(v_loss),
            "Valid Accuracy: {:>3.5f}".format(v_accuracy)
        )
            
            

Epoch  1/10... Batch Loss: 0.02586... Batch Accuracy: 0.95312... Valid Loss: 0.01788... Valid Accuracy: 0.97720
Epoch  2/10... Batch Loss: 0.01115... Batch Accuracy: 0.98438... Valid Loss: 0.01169... Valid Accuracy: 0.98440
Epoch  3/10... Batch Loss: 0.00418... Batch Accuracy: 1.00000... Valid Loss: 0.00924... Valid Accuracy: 0.98780
Epoch  4/10... Batch Loss: 0.00507... Batch Accuracy: 0.98438... Valid Loss: 0.00844... Valid Accuracy: 0.99000
Epoch  5/10... Batch Loss: 0.01592... Batch Accuracy: 0.96875... Valid Loss: 0.00905... Valid Accuracy: 0.98820
Epoch  6/10... Batch Loss: 0.00454... Batch Accuracy: 0.98438... Valid Loss: 0.00879... Valid Accuracy: 0.98920
Epoch  7/10... Batch Loss: 0.00079... Batch Accuracy: 1.00000... Valid Loss: 0.00832... Valid Accuracy: 0.99020
Epoch  8/10... Batch Loss: 0.00007... Batch Accuracy: 1.00000... Valid Loss: 0.00957... Valid Accuracy: 0.99000
Epoch  9/10... Batch Loss: 0.00179... Batch Accuracy: 1.00000... Valid Loss: 0.00990... Valid Accuracy: 